# 크롤링 프로젝트 - 트위터

https://pypi.org/project/GetOldTweets3/#description

## 생각을 하자 생각을!!!!!!!!!!!!!!

1. 트윗 수집 : GetOldTweets
    - 어떤 데이터 형태로 오는지 파악.
    - 어느 정도로 수집해야 서버에서 안 쫓겨 나는지 파악.
    - 데이터 수집 시 시간 얼마나 걸리는지 파악.
    - 검색 조건 하나씩 맞춰 가기 = 검색어, 해시태그, 이모지 등.
    - 수집한 데이터는 일단 다 저장해 놓기.
    

2. 파일 형태로 저장 : 1에서 받은 메타데이터 어떻게 변환할지.
    - 조에서 저장하기로 한 형식, 컬럼 확인.
    - 




3. 파이참으로 코드 변환 및 클래스화 --> 강사님 질문 나중에.

In [25]:
import GetOldTweets3 as got
from bs4 import BeautifulSoup

import datetime
import time
import os
from random import uniform
from tqdm import tqdm_notebook as tn
import csv

1. 트윗 수집 : GetOldTweets
    - 어떤 데이터 형태로 오는지 파악.
    - 어느 정도로 수집해야 서버에서 안 쫓겨 나는지 파악.
    - 데이터 수집 시 시간 얼마나 걸리는지 파악.
    - 검색 조건 하나씩 맞춰 가기 = 검색어, 해시태그, 이모지 등.
    - 수집한 데이터는 일단 다 저장해 놓기.
    

2. 파일 형태로 저장 : 1에서 받은 메타데이터 어떻게 변환할지.
    - 조에서 저장하기로 한 형식, 컬럼 확인.
    - 




3. 파이참으로 코드 변환 및 클래스화 --> 강사님 질문 나중에.

# 멍ㅇㅁ람이람ㅇㄹ 생각을 하자 생각을!!!!!!!!!!!!!!

### 1. 트윗 수집

- 검색어 먼저 한 후(2020 --> 2015까지...)
- 해시태그로 하자.
- 시간이 얼마나 걸리나 일단 보자...

In [114]:
# 날짜 다시 입력해야 할 때 에러 발생시키자.
class NotValidEndDateError(Exception):
    def __init__(self):
        super().__init__('마지막 검색 날짜를 다시 설정하십시오.')

In [115]:
# GetOldTweets3 패키지의 setUntil 메소드는 마지막 날짜 포함하지 않으므로, 주의.
def set_crawl_date(start_date, end_date):
    
    start_date = datetime.datetime.strptime(str(start_date), "%Y%m%d")
    end_date = datetime.datetime.strptime(str(end_date), "%Y%m%d") - datetime.timedelta(days=1)
    
    if end_date == start_date:
        raise NotValidEndDateError
    
    else:   
        print("트윗 수집 날짜 설정: {0}부터 {1}까지".format(start_date, end_date))    
        return start_date.strftime("%Y-%m-%d"), end_date.strftime("%Y-%m-%d")

In [116]:
import sys
import urllib
import json

def getJsonResponse(tweetCriteria, refreshCursor, cookieJar, proxy, useragent=None, debug=False):
    """
    Invoke an HTTP query to Twitter.
    Should not be used as an API function. A static method.
    """
    url = "https://twitter.com/i/search/timeline?"

    if not tweetCriteria.topTweets:
        url += "f=tweets&"

    url += ("vertical=news&q=%s&src=typd&%s"
            "&include_available_features=1&include_entities=1&max_position=%s"
            "&reset_error_state=false")

    urlGetData = ''

    if hasattr(tweetCriteria, 'querySearch'):
        urlGetData += tweetCriteria.querySearch

    if hasattr(tweetCriteria, 'excludeWords'):
        urlGetData += ' -'.join([''] + tweetCriteria.excludeWords)

    if hasattr(tweetCriteria, 'username'):
        if not hasattr(tweetCriteria.username, '__iter__'):
            tweetCriteria.username = [tweetCriteria.username]

        usernames_ = [u.lstrip('@') for u in tweetCriteria.username if u]
        tweetCriteria.username = {u.lower() for u in usernames_ if u}

        usernames = [' from:'+u for u in sorted(tweetCriteria.username)]
        if usernames:
            urlGetData += ' OR'.join(usernames)

    if hasattr(tweetCriteria, 'within'):
        if hasattr(tweetCriteria, 'near'):
            urlGetData += ' near:"%s" within:%s' % (tweetCriteria.near, tweetCriteria.within)
        elif hasattr(tweetCriteria, 'lat') and hasattr(tweetCriteria, 'lon'):
            urlGetData += ' geocode:%f,%f,%s' % (tweetCriteria.lat, tweetCriteria.lon, tweetCriteria.within)

    if hasattr(tweetCriteria, 'since'):
        urlGetData += ' since:' + tweetCriteria.since

    if hasattr(tweetCriteria, 'until'):
        urlGetData += ' until:' + tweetCriteria.until

    if hasattr(tweetCriteria, 'minReplies'):
        urlGetData += ' min_replies:' + tweetCriteria.minReplies

    if hasattr(tweetCriteria, 'minFaves'):
        urlGetData += ' min_faves:' + tweetCriteria.minFaves

    if hasattr(tweetCriteria, 'minRetweets'):
        urlGetData += ' min_retweets:' + tweetCriteria.minRetweets

    if hasattr(tweetCriteria, 'lang'):
        urlLang = 'l=' + tweetCriteria.lang + '&'
    else:
        urlLang = ''
    url = url % (urllib.parse.quote(urlGetData.strip()), urlLang, urllib.parse.quote(refreshCursor))
    useragent = useragent or TweetManager.user_agents[0]

    headers = [
        ('Host', "twitter.com"),
        ('User-Agent', useragent),
        ('Accept', "application/json, text/javascript, */*; q=0.01"),
        ('Accept-Language', "en-US,en;q=0.5"),
        ('X-Requested-With', "XMLHttpRequest"),
        ('Referer', url),
        ('Connection', "keep-alive")
    ]

    if proxy:
        opener = urllib.request.build_opener(urllib.request.ProxyHandler({'http': proxy, 'https': proxy}), urllib.request.HTTPCookieProcessor(cookieJar))
    else:
        opener = urllib.request.build_opener(urllib.request.HTTPCookieProcessor(cookieJar))
    opener.addheaders = headers

    if debug:
        print(url)
        # print('\n'.join(h[0]+': '+h[1] for h in headers))

    try:
        response = opener.open(url)
        jsonResponse = response.read()
    except TimeoutError as e:
        print("Timeout error")
        from time import sleep
        print('sleep 30')
        sleep(30)
        response = opener.open(url)
        jsonResponse = response.read()
    except Exception as e:
        print("An error occured during an HTTP request:", str(e))
        print("Try to open in browser: https://twitter.com/search?q=%s&src=typd" % urllib.parse.quote(urlGetData))
        sys.exit()

    try:
        s_json = jsonResponse.decode()
    except:
        print("Invalid response from Twitter")
        sys.exit()

    try:
        dataJson = json.loads(s_json)
    except:
        print("Error parsing JSON: %s" % s_json)
        sys.exit()

#     if debug:
#         print(s_json)
#         print("---\n")

    return dataJson

In [117]:
def crawl_tweets(start_date, end_date):    
    got.manager.TweetManager.getJsonResponse = getJsonResponse # 함수에 이 부분 넣어주고
    
    print("========== 트윗 수집 시작: {0} ~ {1} ==========".format(start_date, end_date))
    start_time = time.time()
    tweet_criteria = got.manager.TweetCriteria().setQuerySearch('Elon Musk')\
                                                .setSince(start_date)\
                                                .setUntil(end_date)\
                                                .setLang('en')
    tweets = got.manager.TweetManager.getTweets(tweet_criteria, debug=True)
    
    elapsed_time = time.time()-start_time
    
    print("수집 완료 : {}".format(time.strftime("%H:%M:%S", time.gmtime(elapsed_time))))
    print("총 수집 트윗 개수 : {0}".format(len(test_tweet)))
    
    return tweets

In [118]:
def get_results(tweet_data):
    results = []
    for tweet in tweet_data:
        results.append({'url': tweet.permalink,
                        'date': tweet.date,
                        'text': tweet.text,
                        'user': tweet.username,
                        'mentions': tweet.mentions,
                        'retweets': tweet.retweets,
                        'favorites': tweet.favorites,
                        'hashtags': tweet.hashtags})
        
    return results        

In [119]:
def save_tweets(tweet_lists):
    base_file_dir = "tweets"

    if not os.path.exists(base_file_dir):
        os.makedirs(base_file_dir)
        
    with open(f"{base_file_dir}/tweets_{crawl_start}_{crawl_end}.csv", "a", -1, encoding="utf-8") as f:    
        writer = csv.writer(f)
        writer.writerow(['url', 'date', 'text', 'user', 'mentions', 'retweets', 'favorites', 'hashtags'])        
        for tweet_list in tweet_lists:
            writer.writerow(list(tweet_list.values()))
            
    return   

In [120]:
crawl_start, crawl_end = set_crawl_date(20200602, 20200604)

트윗 수집 날짜 설정: 2020-06-02 00:00:00부터 2020-06-03 00:00:00까지


In [121]:
crawl_tweets(crawl_start, crawl_end)

========== 트윗 수집 시작: 2020-06-02 ~ 2020-06-03 ==========
https://twitter.com/i/search/timeline?f=tweets&vertical=news&q=Elon%20Musk%20since%3A2020-06-02%20until%3A2020-06-03&src=typd&l=en&&include_available_features=1&include_entities=1&max_position=&reset_error_state=false
Host: twitter.com
User-Agent: Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36
Accept: application/json, text/javascript, */*; q=0.01
Accept-Language: en-US,en;q=0.5
X-Requested-With: XMLHttpRequest
Referer: https://twitter.com/i/search/timeline?f=tweets&vertical=news&q=Elon%20Musk%20since%3A2020-06-02%20until%3A2020-06-03&src=typd&l=en&&include_available_features=1&include_entities=1&max_position=&reset_error_state=false
Connection: keep-alive
https://twitter.com/i/search/timeline?f=tweets&vertical=news&q=Elon%20Musk%20since%3A2020-06-02%20until%3A2020-06-03&src=typd&l=en&&include_available_features=1&include_entities=1&max_position=thGAVUV0VFVBaGgLyp-

https://twitter.com/i/search/timeline?f=tweets&vertical=news&q=Elon%20Musk%20since%3A2020-06-02%20until%3A2020-06-03&src=typd&l=en&&include_available_features=1&include_entities=1&max_position=thGAVUV0VFVBaCgKeJ9ITcmCMWisC-vf7R3ZgjEjUAFQAlAFUAFQAA&reset_error_state=false
Host: twitter.com
User-Agent: Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36
Accept: application/json, text/javascript, */*; q=0.01
Accept-Language: en-US,en;q=0.5
X-Requested-With: XMLHttpRequest
Referer: https://twitter.com/i/search/timeline?f=tweets&vertical=news&q=Elon%20Musk%20since%3A2020-06-02%20until%3A2020-06-03&src=typd&l=en&&include_available_features=1&include_entities=1&max_position=thGAVUV0VFVBaCgKeJ9ITcmCMWisC-vf7R3ZgjEjUAFQAlAFUAFQAA&reset_error_state=false
Connection: keep-alive
https://twitter.com/i/search/timeline?f=tweets&vertical=news&q=Elon%20Musk%20since%3A2020-06-02%20until%3A2020-06-03&src=typd&l=en&&include_available_features=1

https://twitter.com/i/search/timeline?f=tweets&vertical=news&q=Elon%20Musk%20since%3A2020-06-02%20until%3A2020-06-03&src=typd&l=en&&include_available_features=1&include_entities=1&max_position=thGAVUV0VFVBaEwKPBmJjamCMWisC-vf7R3ZgjEjUAFQAlAFUAFQAA&reset_error_state=false
Host: twitter.com
User-Agent: Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36
Accept: application/json, text/javascript, */*; q=0.01
Accept-Language: en-US,en;q=0.5
X-Requested-With: XMLHttpRequest
Referer: https://twitter.com/i/search/timeline?f=tweets&vertical=news&q=Elon%20Musk%20since%3A2020-06-02%20until%3A2020-06-03&src=typd&l=en&&include_available_features=1&include_entities=1&max_position=thGAVUV0VFVBaEwKPBmJjamCMWisC-vf7R3ZgjEjUAFQAlAFUAFQAA&reset_error_state=false
Connection: keep-alive
https://twitter.com/i/search/timeline?f=tweets&vertical=news&q=Elon%20Musk%20since%3A2020-06-02%20until%3A2020-06-03&src=typd&l=en&&include_available_features=1

https://twitter.com/i/search/timeline?f=tweets&vertical=news&q=Elon%20Musk%20since%3A2020-06-02%20until%3A2020-06-03&src=typd&l=en&&include_available_features=1&include_entities=1&max_position=thGAVUV0VFVBaCwLXJ7sbYmCMWisC-vf7R3ZgjEjUAFQAlAFUAFQAA&reset_error_state=false
Host: twitter.com
User-Agent: Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36
Accept: application/json, text/javascript, */*; q=0.01
Accept-Language: en-US,en;q=0.5
X-Requested-With: XMLHttpRequest
Referer: https://twitter.com/i/search/timeline?f=tweets&vertical=news&q=Elon%20Musk%20since%3A2020-06-02%20until%3A2020-06-03&src=typd&l=en&&include_available_features=1&include_entities=1&max_position=thGAVUV0VFVBaCwLXJ7sbYmCMWisC-vf7R3ZgjEjUAFQAlAFUAFQAA&reset_error_state=false
Connection: keep-alive
https://twitter.com/i/search/timeline?f=tweets&vertical=news&q=Elon%20Musk%20since%3A2020-06-02%20until%3A2020-06-03&src=typd&l=en&&include_available_features=1

https://twitter.com/i/search/timeline?f=tweets&vertical=news&q=Elon%20Musk%20since%3A2020-06-02%20until%3A2020-06-03&src=typd&l=en&&include_available_features=1&include_entities=1&max_position=thGAVUV0VFVBaCwLXlkffWmCMWisC-vf7R3ZgjEjUAFQAlAFUAFQAA&reset_error_state=false
Host: twitter.com
User-Agent: Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36
Accept: application/json, text/javascript, */*; q=0.01
Accept-Language: en-US,en;q=0.5
X-Requested-With: XMLHttpRequest
Referer: https://twitter.com/i/search/timeline?f=tweets&vertical=news&q=Elon%20Musk%20since%3A2020-06-02%20until%3A2020-06-03&src=typd&l=en&&include_available_features=1&include_entities=1&max_position=thGAVUV0VFVBaCwLXlkffWmCMWisC-vf7R3ZgjEjUAFQAlAFUAFQAA&reset_error_state=false
Connection: keep-alive
https://twitter.com/i/search/timeline?f=tweets&vertical=news&q=Elon%20Musk%20since%3A2020-06-02%20until%3A2020-06-03&src=typd&l=en&&include_available_features=1

...

https://twitter.com/i/search/timeline?f=tweets&vertical=news&q=Elon%20Musk%20since%3A2020-06-02%20until%3A2020-06-03&src=typd&l=en&&include_available_features=1&include_entities=1&max_position=thGAVUV0VFVBaCwL2Zz8HAlyMWisC-vf7R3ZgjEjUAFQAlAFUAFQAA&reset_error_state=false
Host: twitter.com
User-Agent: Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36
Accept: application/json, text/javascript, */*; q=0.01
Accept-Language: en-US,en;q=0.5
X-Requested-With: XMLHttpRequest
Referer: https://twitter.com/i/search/timeline?f=tweets&vertical=news&q=Elon%20Musk%20since%3A2020-06-02%20until%3A2020-06-03&src=typd&l=en&&include_available_features=1&include_entities=1&max_position=thGAVUV0VFVBaCwL2Zz8HAlyMWisC-vf7R3ZgjEjUAFQAlAFUAFQAA&reset_error_state=false
Connection: keep-alive
https://twitter.com/i/search/timeline?f=tweets&vertical=news&q=Elon%20Musk%20since%3A2020-06-02%20until%3A2020-06-03&src=typd&l=en&&include_available_features=1

https://twitter.com/i/search/timeline?f=tweets&vertical=news&q=Elon%20Musk%20since%3A2020-06-02%20until%3A2020-06-03&src=typd&l=en&&include_available_features=1&include_entities=1&max_position=thGAVUV0VFVBaAwLX916O-lyMWisC-vf7R3ZgjEjUAFQAlAFUAFQAA&reset_error_state=false
Host: twitter.com
User-Agent: Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36
Accept: application/json, text/javascript, */*; q=0.01
Accept-Language: en-US,en;q=0.5
X-Requested-With: XMLHttpRequest
Referer: https://twitter.com/i/search/timeline?f=tweets&vertical=news&q=Elon%20Musk%20since%3A2020-06-02%20until%3A2020-06-03&src=typd&l=en&&include_available_features=1&include_entities=1&max_position=thGAVUV0VFVBaAwLX916O-lyMWisC-vf7R3ZgjEjUAFQAlAFUAFQAA&reset_error_state=false
Connection: keep-alive
https://twitter.com/i/search/timeline?f=tweets&vertical=news&q=Elon%20Musk%20since%3A2020-06-02%20until%3A2020-06-03&src=typd&l=en&&include_available_features=1

https://twitter.com/i/search/timeline?f=tweets&vertical=news&q=Elon%20Musk%20since%3A2020-06-02%20until%3A2020-06-03&src=typd&l=en&&include_available_features=1&include_entities=1&max_position=thGAVUV0VFVBaIwL2x0fO7lyMWisC-vf7R3ZgjEjUAFQAlAFUAFQAA&reset_error_state=false
Host: twitter.com
User-Agent: Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36
Accept: application/json, text/javascript, */*; q=0.01
Accept-Language: en-US,en;q=0.5
X-Requested-With: XMLHttpRequest
Referer: https://twitter.com/i/search/timeline?f=tweets&vertical=news&q=Elon%20Musk%20since%3A2020-06-02%20until%3A2020-06-03&src=typd&l=en&&include_available_features=1&include_entities=1&max_position=thGAVUV0VFVBaIwL2x0fO7lyMWisC-vf7R3ZgjEjUAFQAlAFUAFQAA&reset_error_state=false
Connection: keep-alive
https://twitter.com/i/search/timeline?f=tweets&vertical=news&q=Elon%20Musk%20since%3A2020-06-02%20until%3A2020-06-03&src=typd&l=en&&include_available_features=1

https://twitter.com/i/search/timeline?f=tweets&vertical=news&q=Elon%20Musk%20since%3A2020-06-02%20until%3A2020-06-03&src=typd&l=en&&include_available_features=1&include_entities=1&max_position=thGAVUV0VFVBaAwKvN2Ou5lyMWisC-vf7R3ZgjEjUAFQAlAFUAFQAA&reset_error_state=false
Host: twitter.com
User-Agent: Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36
Accept: application/json, text/javascript, */*; q=0.01
Accept-Language: en-US,en;q=0.5
X-Requested-With: XMLHttpRequest
Referer: https://twitter.com/i/search/timeline?f=tweets&vertical=news&q=Elon%20Musk%20since%3A2020-06-02%20until%3A2020-06-03&src=typd&l=en&&include_available_features=1&include_entities=1&max_position=thGAVUV0VFVBaAwKvN2Ou5lyMWisC-vf7R3ZgjEjUAFQAlAFUAFQAA&reset_error_state=false
Connection: keep-alive
https://twitter.com/i/search/timeline?f=tweets&vertical=news&q=Elon%20Musk%20since%3A2020-06-02%20until%3A2020-06-03&src=typd&l=en&&include_available_features=1

 ...]

#### 테스트

* 목적 : 내가 보는 트위터 데이터랑 맞게 가져오는지

* 기간 설정 : 20200601
    - 근데, 가장 위에 있는게 20200605가 나온다...

* Tweet 객체
    - id : 뭐지?
    - permalink : url. --> 조에서 필요.
    - username : user id
    - text : 내용. --> 조에서 필요.
    - to: 뭐지?
    - retweet, favorites 
    - date : 날짜 --> 조에서 필요.
    - hashtag : 해시태그
    - geo

In [33]:
# 1. 검색어로 가져오기
tweetCriteria = got.manager.TweetCriteria().setQuerySearch('Elon Musk')\
                                            .setSince("2020-06-01")\
                                            .setSince("2020-06-03")\
                                            .setMaxTweets(2)\
                                            .setLang('en')
tweet = got.manager.TweetManager.getTweets(tweetCriteria)

In [34]:
tweet

In [35]:
tweet[0].id

'1268819826367008768'

In [36]:
tweet[0].permalink

'https://twitter.com/SkepticStevo/status/1268819826367008768'

In [37]:
tweet[0].username

'SkepticStevo'

In [20]:
tweet[0].to

In [38]:
tweet[0].text

'Anyone else notice @elonMusk aging really poorly the last couple of years...while @JeffBezos looks younger than his pre amazon days...SOME 26 years ago!!! #Fraudboy $TSLA $AMZN @amazon #TSLAq Hard to keep the fraud going, #Tesla R&amp;D spent mostly on Planning future #FRAUD'

In [15]:
tweet[0].date

datetime.datetime(2020, 6, 5, 5, 54, 17, tzinfo=datetime.timezone.utc)

In [16]:
tweet[0].retweets

0

In [17]:
tweet[0].favorites

0

In [18]:
tweet[0].mentions

'@elonmusk'

In [21]:
tweet[0].hashtags

''

In [22]:
tweet[0].geo

''

In [23]:
tweet[1].to  # 답장인 듯?

'elonmusk'

In [24]:
tweet[1].text

'When is Tesla going to introduce mobile homes?'

In [25]:
tweet[1].permalink

'https://twitter.com/yasirwattu/status/1268783175628730368'